viernes 03 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 2 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de un perceptrón multicapa o red neuronal de múltiples capas (Multi-Layer Neural Network). En primera oportunidad, se utilizará el perceptrón para clasificar y emular compuertas lógicas AND, OR y XOR, tanto de 2 como de 4 entradas. Luego, se realizará una clasificación trabajando con el conjunto de datos [wine](https://archive.ics.uci.edu/ml/datasets/wine) provenientes de la [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) que contiene 13 atributos en total y que intentan describir clases de vinos provenientes de 3 cultivos distintos.


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **Numpy:** Herramienta de computación científica, que nos permitirá trabajar a través de vectores
- **Pandas:** Para la manipulación y lectura de datos
- **Matplotlib:** Para la visualización gráfica de diversos datos de interés
- **Itertools:** Como herramienta para iteración de objetos
- **Scikit-learn:** Para la obtención del dataset _wine_ y el uso de herramientas de evaluación de desempeño para los perceptrones multicapa.

In [18]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import itertools as its
import sklearn as skl
from sklearn.datasets import load_wine

#### Definición de la clase TruthTableGenerator
---

La clase TruthTableGenerator permite generar tablas de verdad para 3 tipos de compuertas lógicas: AND, OR y XOR.

En cuanto a las funciones definidas, se tiene:

| **Función**  | **Descripción**  |
| ------------ | ------------ |
| `generate_table(n_inputs, logic_gate)`  |  Genera una tabla de verdad de n-variables (*n_inputs*), basada en una compuerta lógica determinada (*logic_gate*)|

In [19]:
class TruthTableGenerator():
    
    def generate_table(n_inputs, logic_gate):
        table = its.product([0,1], repeat = n_inputs)
        table = pd.DataFrame(table)
        results = []
        
        if logic_gate == 'AND':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( all(row) )
            
        elif logic_gate == 'OR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( any(row) )
                    
        elif logic_gate == 'XOR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                tmp_result = row[0] ^ row[1]
                for j in range(2, n_inputs):
                    tmp_result = tmp_result ^ row[j]
                results.append( tmp_result )
        
        else:
            return None
        
        table['result'] = results
        return table

#### Definición de la clase Layer
---
La clase Layer permite representar una de las capas que conforma al Perceptrón Multicapa. Requiere de 3 parámetros:

| <p style='text-align: left;'>**Parámetro**</p> | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `number_of_inputs` |  <p style='text-align: justify;'>Valor númerico entero. Define la cantidad de entradas que recibe la capa, la cual debe ser mayor que 0. Por defecto, se define el número de entradas como 1.</p>|
| `number_of_neurons` | <p style='text-align: justify;'>Valor numérico entero. Define la cantidad de neuronas que posee la capa, la cual debe ser mayor que 0. Por defecto, se define el número de neuronas como 1.</p> |
| `activation_function` | <p style='text-align: justify;'>Cadena de caracteres. Define la función de activación que empleará la capa, la cual puede ser sigmoide (_sigmoid_) o ReLU (_relu_).</p> |

En cuanto a las funciones definidas, se tiene:

| <p style='text-align: left;'>**Función**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `calculate_output(input_vector)` |  <p style='text-align: justify;'>Realiza una predicción para un conjunto de datos (*input_vector*), empleando los valores actuales de los pesos (*weights*) asociados a la capa.</p>|


In [48]:
class Layer:
    def __init__(self,
                 number_of_inputs = 1,
                 number_of_neurons = 1,
                 activation_function = 'sigmoid'):
        self.number_of_neurons = number_of_neurons
        self.number_of_inputs = number_of_inputs
        self.activation_function = activation_function
        self.weights = np.random.rand(self.number_of_inputs + 1, self.number_of_neurons)
        self.input = None
        self.output = None
        
    def calculate_output(self, input_vector):
        self.input = np.append(input_vector, 1)
        sum_ = np.matmul(self.input, self.weights)
        if self.activation_function == 'sigmoid': self.output = sigmoid(sum_)
        elif self.activation_function == 'relu': self.output = relu(sum_)
        else: self.output = None
        return self.output
        
            

class MLP:
    def __init__(self,
                 data = None,
                 number_of_inputs = 1,
                 number_of_hidden_layers = 0,
                 number_of_neurons_for_each_layer = None,
                 activation_functions_for_each_layer = None):
        
        self.data = data
        self.data_nrows, self.data_ncols = data.shape
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_neurons_for_each_layer = number_of_neurons_for_each_layer
        self.activation_functions_for_each_layer = activation_functions_for_each_layer
        self.layers = []
        
        # Primero se crea una capa escondida (hidden layer) según la cantidad de entradas
        # definidas para el perceptrón. Si no se define un número de capas escondidas,
        # entonces solamente se creará la capa de salida (output layer), recreando a un
        # perceptrón simple.
        
        self.layers.append(
            Layer(number_of_inputs = number_of_inputs,
                  number_of_neurons = number_of_neurons_for_each_layer[0],
                  activation_function = activation_functions_for_each_layer[0])
        )
        for i in range(1, self.number_of_hidden_layers + 1):
            self.layers.append(
                Layer(number_of_inputs = number_of_neurons_for_each_layer[i-1],
                      number_of_neurons = number_of_neurons_for_each_layer[i],
                      activation_function = activation_functions_for_each_layer[i])
            )
    
    def training(self, number_of_iterations):
        errors_series = []
        for i in range(number_of_iterations):
            errors = 0
            for j in range(self.data_nrows):
                row = np.array(self.data.loc[[j]].values[0])
                predicted_value = self.predict( row[:-1], row[-1] )
                error = row[-1] - predicted_value
                errors += int(error != 0.0)
            errors_series.append(errors)
        return np.array(errors_series)
    
    def predict(self, input_vector, target_vector):
        output_vector = None
        for layer in self.layers:
            output_vector = layer.calculate_output(input_vector)
            input_vector = output_vector
        self.adjust_weights(target_vector)
        return output_vector
    
    def adjust_weights(self, target_vector):
        flag = True
        errors = []
        deltas_values = []
        
        # Calculo del error para la capa de salida
        error = np.subtract(target_vector, self.layers[-1].output)
        
        #(f(a') - dt)*f(a')(1-f(a'))*outputs_hidden
        delta = np.array(error * sigmoid_derivative(self.layers[-1].output)*self.layers[-2].output)
        
        errors.append(error)
        deltas_values.append(delta)
                                                        #Sin contar la de salida
        for i in list(reversed(range(len(self.layers))))[1:]:            
            delta_hidden = error * sigmoid_derivative(self.layers[-1].output)
            for j in range(len(self.layers)-i-1):
                delta_hidden = delta_hidden * self.layers[i+j+1].weights*sigmoid_derivative(self.layers[i].output)
                for k in range(len(delta_hidden)):
                    delta_hidden[k] = delta_hidden[k]*self.layers[i].input[k]
            deltas_values.append(delta_hidden)
        
            
                
        
        deltas_values.reverse()
        for i in range(len(self.layers)):
                        #nu
            adjustment = 0.1*-1*deltas_values[i]
            print(self.layers[i].weights[:len(self.layers[i].weights)-1])
            for j in range(len(self.layers[i].weights[:len(self.layers[i].weights)-1])):
                    self.layers[i].weights[j] += adjustment[j] 
            
            
    
    """
    def adjust_weights(self, target_vector):
        flag = True
        errors = []
        deltas_values = []
        
        # Calculo del error para la capa de salida
        error = np.subtract(target_vector, self.layers[-1].output)
        delta = np.array(error * sigmoid_derivative(self.layers[-1].output))
        errors.append(error)
        deltas_values.append(delta)
        
        for i in list(reversed(range(len(self.layers))))[1:]:
            print(i)
            error = np.dot(deltas_values[-1], np.transpose(self.layers[i+1].weights))
            print("error:", error)
            print("sigmoid_derivative:", sigmoid_derivative(self.layers[i+1].input))
            delta = np.array(
                error * sigmoid_derivative(self.layers[i+1].input)
            )
            deltas_values.append(delta)
        
        deltas_values.reverse()
        print("delta_values:", deltas_values)
        for i in range(len(self.layers)):
            
            print("== ADJUSTMENT ==")
            print("np.transpose([self.layers[i].input])", np.transpose([self.layers[i].input]) )
            print("deltas_values[i]:", deltas_values[i])
            adjustment = np.dot([self.layers[i].input], np.transpose([deltas_values[i]]))
            self.layers[i].weights += adjustment
        
    """
            
        

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 -  sigmoid(x))

def relu(x):
    return np.maximum(0, x)
                
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

    

a = sklearn_to_df(load_wine())

b = TruthTableGenerator.generate_table(n_inputs=4, logic_gate='XOR')
perceptron = MLP(data = b,
                 number_of_inputs=4,
                 number_of_hidden_layers=1,
                 number_of_neurons_for_each_layer=[3,1],
                 activation_functions_for_each_layer=['sigmoid', 'sigmoid'])

perceptron.training(20)

[[0.90279316 0.83512607 0.78240755]
 [0.2646084  0.6226632  0.87354565]
 [0.92173863 0.93441097 0.64241656]
 [0.50792185 0.29105502 0.81231611]]
[[0.56185858]
 [0.23150432]
 [0.37660798]]
[[0.90279316 0.83512607 0.78240755]
 [0.2646084  0.6226632  0.87354565]
 [0.92173863 0.93441097 0.64241656]
 [0.50792185 0.29105502 0.81231611]]
[[0.57113255]
 [0.24142291]
 [0.38807981]]
[[0.90279316 0.83512607 0.78240755]
 [0.2646084  0.6226632  0.87354565]
 [0.92173863 0.93441097 0.64241656]
 [0.50729983 0.29043002 0.81172781]]
[[0.56897106]
 [0.23931202]
 [0.38539433]]
[[0.90279316 0.83512607 0.78240755]
 [0.2646084  0.6226632  0.87354565]
 [0.9214764  0.93415155 0.64215856]
 [0.50729983 0.29043002 0.81172781]]
[[0.56666311]
 [0.23691095]
 [0.38294715]]
[[0.90279316 0.83512607 0.78240755]
 [0.2646084  0.6226632  0.87354565]
 [0.92293673 0.93561642 0.64358051]
 [0.51049211 0.29363219 0.81483615]]
[[0.58154542]
 [0.25165014]
 [0.39901427]]
[[0.90279316 0.83512607 0.78240755]
 [0.26442003 0.6224815  

[[0.94769708 0.87972878 0.82616734]
 [0.29316356 0.65067934 0.90077467]
 [0.95929666 0.97168534 0.6791338 ]
 [0.56214983 0.34505145 0.86445681]]
[[1.02166944]
 [0.70728673]
 [0.89176923]]
[[0.94769708 0.87972878 0.82616734]
 [0.29606484 0.65350079 0.90355712]
 [0.96295469 0.97524271 0.68264201]
 [0.56214983 0.34505145 0.86445681]]
[[1.03700034]
 [0.72397292]
 [0.90909602]]
[[0.94769708 0.87972878 0.82616734]
 [0.29595491 0.6533922  0.90345097]
 [0.96281665 0.97510635 0.68250871]
 [0.56202272 0.34492589 0.86433406]]
[[1.0363598 ]
 [0.72331091]
 [0.90839582]]
[[0.94746015 0.87949315 0.82593641]
 [0.29595491 0.6533922  0.90345097]
 [0.96281665 0.97510635 0.68250871]
 [0.56202272 0.34492589 0.86433406]]
[[1.03557782]
 [0.72251303]
 [0.90754148]]
[[0.95165093 0.88371565 0.82999939]
 [0.29595491 0.6533922  0.90345097]
 [0.96281665 0.97510635 0.68250871]
 [0.56541036 0.34833918 0.86761839]]
[[1.05165765]
 [0.73822335]
 [0.9250656 ]]
[[0.95583758 0.88788624 0.83416482]
 [0.29595491 0.6533922  

[[1.02573573 0.95729249 0.90238655]
 [0.34883229 0.70533928 0.95420521]
 [1.03115125 1.04298685 0.74952114]
 [0.61174637 0.39453688 0.91246279]]
[[1.49102915]
 [1.19129639]
 [1.40962026]]
[[1.02573573 0.95729249 0.90238655]
 [0.35389746 0.71031117 0.95892749]
 [1.03115125 1.04298685 0.74952114]
 [0.61530563 0.39803059 0.91578111]]
[[1.50563269]
 [1.20688312]
 [1.42767973]]
[[1.02573573 0.95729249 0.90238655]
 [0.35890259 0.71518846 0.96374358]
 [1.03707206 1.04875643 0.75521831]
 [0.61530563 0.39803059 0.91578111]]
[[1.52165302]
 [1.22416083]
 [1.44554264]]
[[1.02573573 0.95729249 0.90238655]
 [0.35885857 0.71514491 0.96370086]
 [1.03702008 1.048705   0.75516788]
 [0.61527553 0.39800081 0.9157519 ]]
[[1.52149675]
 [1.22400018]
 [1.44537429]]
[[1.02563306 0.95719035 0.90228639]
 [0.35885857 0.71514491 0.96370086]
 [1.03702008 1.048705   0.75516788]
 [0.61527553 0.39800081 0.9157519 ]]
[[1.52126021]
 [1.22375914]
 [1.4451173 ]]
[[1.03185777 0.96345924 0.90833754]
 [0.35885857 0.71514491 

[[1.14561392 1.0766405  1.01978469]
 [0.45653212 0.81119977 1.05767425]
 [1.14787284 1.15877687 0.86402938]
 [0.67306066 0.4556461  0.97176047]]
[[2.04956356]
 [1.76625275]
 [2.02309576]]
[[1.15382058 1.08482388 1.0279617 ]
 [0.45653212 0.81119977 1.05767425]
 [1.15597352 1.16685457 0.87210079]
 [0.67306066 0.4556461  0.97176047]]
[[2.06760712]
 [1.78442578]
 [2.04130417]]
[[1.15380832 1.08481159 1.02794952]
 [0.45653212 0.81119977 1.05767425]
 [1.15596142 1.16684244 0.87208877]
 [0.6730557  0.45564112 0.97175554]]
[[2.06757881]
 [1.78439763]
 [2.04127544]]
[[1.16232264 1.09316192 1.03615251]
 [0.46388029 0.81840641 1.06475374]
 [1.15596142 1.16684244 0.87208877]
 [0.6730557  0.45564112 0.97175554]]
[[2.08428859]
 [1.80203585]
 [2.0597274 ]]
[[1.16230941 1.09314878 1.03613961]
 [0.46386885 0.81839505 1.06474259]
 [1.15596142 1.16684244 0.87208877]
 [0.67305039 0.45563585 0.97175036]]
[[2.08425998]
 [1.80200677]
 [2.059697  ]]
[[1.16229777 1.09313723 1.0361281 ]
 [0.46385879 0.81838506 

[[1.32341334 1.25363176 1.19472185]
 [0.60720627 0.95981976 1.20372331]
 [1.31527386 1.32529852 1.02918606]
 [0.73060812 0.5130449  1.02782495]]
[[2.64331284]
 [2.37435964]
 [2.6637733 ]]
[[1.32341334 1.25363176 1.19472185]
 [0.60720627 0.95981976 1.20372331]
 [1.31526556 1.32529029 1.02917786]
 [0.73060812 0.5130449  1.02782495]]
[[2.64330099]
 [2.37434745]
 [2.66376095]]
[[1.32341334 1.25363176 1.19472185]
 [0.60720627 0.95981976 1.20372331]
 [1.32603412 1.33608332 1.03976407]
 [0.73399226 0.51643673 1.03115179]]
[[2.66095652]
 [2.39189674]
 [2.68219854]]
[[1.32341334 1.25363176 1.19472185]
 [0.60719475 0.95980863 1.20371256]
 [1.32603412 1.33608332 1.03976407]
 [0.73399226 0.51643673 1.03115179]]
[[2.66094193]
 [2.39188001]
 [2.68217987]]
[[1.32341334 1.25363176 1.19472185]
 [0.61719998 0.96965958 1.21317779]
 [1.32603412 1.33608332 1.03976407]
 [0.73749392 0.51988439 1.03446445]]
[[2.67693067]
 [2.40864395]
 [2.70080461]]
[[1.32341334 1.25363176 1.19472185]
 [0.62697754 0.97926551 

array([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16])

In [4]:
range(3)

range(0, 3)

In [42]:
A = [1,2,3,4]
print(A[:len(A)-1])

[1, 2, 3]
